In [2]:
# !pip install mlflow dagshub catboost optuna

In [3]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, MinMaxScaler, PowerTransformer, OrdinalEncoder
from sklearn.model_selection import train_test_split, KFold
import dagshub
import mlflow
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
import optuna
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [4]:
dagshub.init(repo_owner='Aryanupadhyay23', repo_name='Zomato-Food-Delivery-Time-prediction', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=f30aa4b1-da25-4dc0-8a8a-bb49c64c190a&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=8f629c7b3418c9d36e492be9bcfb0ffb577b4140b2ffe39880015e99ba99bf10




Accessing as Aryanupadhyay23

Initialized MLflow to track repo "Aryanupadhyay23/Zomato-Food-Delivery-Time-prediction"

Repository Aryanupadhyay23/Zomato-Food-Delivery-Time-prediction initialized!

In [5]:
# set the tracking server

mlflow.set_tracking_uri("https://dagshub.com/Aryanupadhyay23/Zomato-Food-Delivery-Time-prediction.mlflow")

In [6]:
# mlflow experiment
mlflow.set_experiment("Stacking Regressor Optimization")

<Experiment: artifact_location='mlflow-artifacts:/5021c5346d0a460880feb942e1cce131', creation_time=1770697631903, experiment_id='9', last_update_time=1770697631903, lifecycle_stage='active', name='Stacking Regressor Optimization', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [7]:
df = pd.read_csv('/kaggle/input/datasets/aryanumri/food-delivery-time-prediction/food_delivery_interim.csv')
df.head()

,rider_age,rider_ratings,weather,traffic_density,vehicle_condition,order_type,vehicle_type,multiple_deliveries,festival,time_taken,city_type,day_name,time_of_day,distance
0,36.0,4.2,fog,jam,2,snack,motorcycle,3.0,no,46,metropolitian,saturday,dinner_peak,10.280582
1,21.0,4.7,stormy,high,1,meal,motorcycle,1.0,no,23,metropolitian,sunday,afternoon,6.242319
2,23.0,4.7,sandstorms,medium,1,drinks,scooter,1.0,no,21,metropolitian,friday,evening_snacks,13.787860
3,34.0,4.3,sandstorms,low,0,buffet,motorcycle,0.0,no,20,metropolitian,sunday,breakfast,2.930258
4,24.0,4.7,fog,jam,1,snack,scooter,1.0,no,41,metropolitian,monday,evening_snacks,19.396618


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38055 entries, 0 to 38054
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   rider_age            38055 non-null  float64
 1   rider_ratings        38055 non-null  float64
 2   weather              38055 non-null  object 
 3   traffic_density      38055 non-null  object 
 4   vehicle_condition    38055 non-null  int64  
 5   order_type           38055 non-null  object 
 6   vehicle_type         38055 non-null  object 
 7   multiple_deliveries  38055 non-null  float64
 8   festival             38055 non-null  object 
 9   time_taken           38055 non-null  int64  
 10  city_type            38055 non-null  object 
 11  day_name             38055 non-null  object 
 12  time_of_day          38055 non-null  object 
 13  distance             38055 non-null  float64
dtypes: float64(4), int64(2), object(8)
memory usage: 4.1+ MB


In [9]:
X = df.drop(columns='time_taken')
y = df['time_taken']

In [10]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [11]:
num_cols = ["rider_age","rider_ratings","distance"]

nominal_cat_cols = ["weather","order_type","vehicle_type","festival","city_type","day_name","time_of_day"]

ordinal_cat_cols = ["traffic_density"]

In [12]:
traffic_order = ["low","medium","high","jam"]

In [13]:
preprocessor = ColumnTransformer(
    transformers=[
        ("scale", StandardScaler(), num_cols),
        (
            "nominal_encoder",
            OneHotEncoder(handle_unknown="ignore", drop="first", sparse_output=False),
            nominal_cat_cols
        ),
        (
            "ordinal_encoder",
            OrdinalEncoder(categories=[traffic_order]),
            ordinal_cat_cols
        )
    ],
    remainder="passthrough",
    n_jobs=-1,
    force_int_remainder_cols=False,
    verbose_feature_names_out=False
)

preprocessor.set_output(transform="pandas")

ColumnTransformer(force_int_remainder_cols=False, n_jobs=-1,
                  remainder='passthrough',
                  transformers=[('scale', StandardScaler(),
                                 ['rider_age', 'rider_ratings', 'distance']),
                                ('nominal_encoder',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore',
                                               sparse_output=False),
                                 ['weather', 'order_type', 'vehicle_type',
                                  'festival', 'city_type', 'day_name',
                                  'time_of_day']),
                                ('ordinal_encoder',
                                 OrdinalEncoder(categories=[['low', 'medium',
                                                             'high', 'jam']]),
                                 ['traffic_density'])],
                  verbose_feature_names_out=False)

In [14]:
pt = PowerTransformer()

y_train_pt = pt.fit_transform(y_train.values.reshape(-1,1))
y_test_pt = pt.transform(y_test.values.reshape(-1,1))

In [15]:
y_train_pt = np.ravel(y_train_pt)
y_test_pt  = np.ravel(y_test_pt)

In [16]:
## pipeline

preprocessing_pipeline = Pipeline(
    steps=[
        ("preprocessor",preprocessor)
    ]
)

preprocessing_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(force_int_remainder_cols=False, n_jobs=-1,
                                   remainder='passthrough',
                                   transformers=[('scale', StandardScaler(),
                                                  ['rider_age', 'rider_ratings',
                                                   'distance']),
                                                 ('nominal_encoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['weather', 'order_type',
                                                   'vehicle_type', 'festival',
                                                   'city_type', 'day_name',
                                                   'time_of_day']),
                                                 ('ordinal_encoder',
                                                  OrdinalEncoder(categories=[['low',
                                                                              'medium',
                                                                              'high',
                                                                              'jam']]),
                                                  ['traffic_density'])],
                                   verbose_feature_names_out=False))])

In [17]:
X_train_trans = preprocessing_pipeline.fit_transform(X_train)
X_test_trans = preprocessing_pipeline.transform(X_test)

X_train_trans

,rider_age,rider_ratings,distance,weather_fog,weather_sandstorms,weather_stormy,weather_sunny,weather_windy,order_type_drinks,order_type_meal,...,day_name_tuesday,day_name_wednesday,time_of_day_breakfast,time_of_day_dinner_peak,time_of_day_evening_snacks,time_of_day_late_night,time_of_day_lunch_peak,traffic_density,vehicle_condition,multiple_deliveries
6965,-0.282097,1.164633,-1.211247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2,0.0
14052,1.454428,0.849370,0.717574,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0,3.0
25717,-0.455749,0.218843,0.144911,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2,1.0
35085,1.280776,-1.988001,-0.385292,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2,1.0
5921,0.759818,-0.726947,0.689886,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16850,-1.671317,1.164633,-1.203960,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,1.0
6265,-0.976707,-1.672737,-0.932219,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0.0
11284,-0.976707,-1.672737,0.653365,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0,1.0
860,1.454428,-1.357474,0.757606,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,2,2.0


In [18]:
# ---------------- RandomForest (CPU) ----------------
best_rf_params = {
    "n_estimators": 290,
    "criterion": "squared_error",
    "max_depth": 14,
    "min_samples_split": 7,
    "min_samples_leaf": 1,
    "max_features": None,
    "bootstrap": True,
    "random_state": 42,
    "n_jobs": -1
}

best_rf = RandomForestRegressor(**best_rf_params)

# ---------------- CatBoost (GPU ENABLED) ----------------
best_cat_params = {
    "iterations": 1466,
    "depth": 10,
    "learning_rate": 0.038080477733221894,
    "l2_leaf_reg": 26.021727151100524,
    "random_strength": 9.561951563676054,
    "bagging_temperature": 0.7315752426106197,
    "loss_function": "RMSE",
    "eval_metric": "RMSE",
    "task_type": "GPU",
    "devices": "0",
    "random_seed": 42,
    "verbose": 0
}

best_cat = CatBoostRegressor(**best_cat_params)

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_absolute_error
import mlflow

def objective(trial):
    with mlflow.start_run(nested=True):

        # ---------------- Meta-model selection ----------------
        meta_model_name = trial.suggest_categorical("model", ["LR", "KNN", "DT"])

        if meta_model_name == "LR":
            meta = LinearRegression()

        elif meta_model_name == "KNN":
            meta = KNeighborsRegressor(
                n_neighbors=trial.suggest_int("n_neighbors_knn", 1, 15),
                weights=trial.suggest_categorical(
                    "weights_knn", ["uniform", "distance"]
                ),
                n_jobs=-1
            )

        else:  # DT
            meta = DecisionTreeRegressor(
                max_depth=trial.suggest_int("max_depth_dt", 1, 10),
                min_samples_split=trial.suggest_int("min_samples_split_dt", 2, 10),
                min_samples_leaf=trial.suggest_int("min_samples_leaf_dt", 1, 10),
                random_state=42
            )

        mlflow.log_param("meta_model", meta_model_name)

        # ---------------- Stacking Regressor ----------------
        stacking_reg = StackingRegressor(
            estimators=[
                ("rf", best_rf),
                ("cat", best_cat)
            ],
            final_estimator=meta,
            cv=5,            
            n_jobs=1,       
            passthrough=False
        )

        model = TransformedTargetRegressor(
            regressor=stacking_reg,
            transformer=pt
        )

        # ---------------- Train ----------------
        model.fit(X_train_trans, y_train)

        # ---------------- Test Evaluation ----------------
        y_pred_test = model.predict(X_test_trans)
        test_mae = mean_absolute_error(y_test, y_pred_test)

        mlflow.log_metric("test_mae", test_mae)

        return test_mae

In [20]:
study = optuna.create_study(direction="minimize")

with mlflow.start_run(run_name="Meta_Model_Selection"):

    study.optimize(
        objective,
        n_trials=20,
        n_jobs=1,                
        show_progress_bar=True
    )

    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_test_mae", study.best_value)

    print("\nOptimization Complete!")
    print(f"Best Test MAE: {study.best_value:.4f}")
    print("Best Parameters:")
    for k, v in study.best_params.items():
        print(f"  {k}: {v}")

[I 2026-02-10 05:50:23,123] A new study created in memory with name: no-name-e926ffac-f8a0-4e3b-a32b-328d576220f8


  0%|          | 0/20 [00:00<?, ?it/s]

🏃 View run fearless-grouse-360 at: https://dagshub.com/Aryanupadhyay23/Zomato-Food-Delivery-Time-prediction.mlflow/#/experiments/9/runs/3bbbae233cc849ac979a71711f91e382
🧪 View experiment at: https://dagshub.com/Aryanupadhyay23/Zomato-Food-Delivery-Time-prediction.mlflow/#/experiments/9
[I 2026-02-10 05:53:44,731] Trial 0 finished with value: 2.9922067303690176 and parameters: {'model': 'LR'}. Best is trial 0 with value: 2.9922067303690176.
🏃 View run melodic-kite-630 at: https://dagshub.com/Aryanupadhyay23/Zomato-Food-Delivery-Time-prediction.mlflow/#/experiments/9/runs/dda867a8ae204e70a2087ea884b68f22
🧪 View experiment at: https://dagshub.com/Aryanupadhyay23/Zomato-Food-Delivery-Time-prediction.mlflow/#/experiments/9
[I 2026-02-10 05:57:08,127] Trial 1 finished with value: 2.9922067303690176 and parameters: {'model': 'LR'}. Best is trial 0 with value: 2.9922067303690176.
🏃 View run bold-snipe-310 at: https://dagshub.com/Aryanupadhyay23/Zomato-Food-Delivery-Time-prediction.mlflow/#/exp

In [21]:
from optuna.visualization import plot_optimization_history, plot_param_importances
import mlflow.sklearn

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import (
    mean_absolute_error,
    r2_score,
    mean_squared_error
)

import numpy as np

# ================================
# 1. Optuna Visualizations
# ================================
plot_optimization_history(study).show()
plot_param_importances(study).show()

# ================================
# 2. Rebuild Best Meta Model
# ================================
best_params = study.best_params
meta_type = best_params["model"]

if meta_type == "LR":
    final_meta = LinearRegression()

elif meta_type == "KNN":
    final_meta = KNeighborsRegressor(
        n_neighbors=best_params.get("n_neighbors_knn"),
        weights=best_params.get("weights_knn"),
        n_jobs=-1
    )

elif meta_type == "DT":
    final_meta = DecisionTreeRegressor(
        max_depth=best_params.get("max_depth_dt"),
        min_samples_split=best_params.get("min_samples_split_dt"),
        min_samples_leaf=best_params.get("min_samples_leaf_dt"),
        random_state=42
    )

# ================================
# 3. Final Stacking Model 
# ================================
final_stacking = StackingRegressor(
    estimators=[
        ("rf", best_rf),
        ("cat", best_cat)   
    ],
    final_estimator=final_meta,
    cv=5,
    n_jobs=1,              
    passthrough=False
)

final_model = TransformedTargetRegressor(
    regressor=final_stacking,
    transformer=pt
)

# ================================
# 4. Train Final Model
# ================================
final_model.fit(X_train_trans, y_train)
print("Final model trained successfully.")

# ================================
# 5. Predictions
# ================================
y_train_pred = final_model.predict(X_train_trans)
y_test_pred = final_model.predict(X_test_trans)

# ================================
# 6. Metrics
# ================================
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

# ================================
# 7. Print Metrics
# ================================
print("\n========== FINAL MODEL PERFORMANCE ==========")
print(f"Train MAE  : {train_mae:.4f}")
print(f"Test  MAE  : {test_mae:.4f}")
print(f"Train RMSE : {train_rmse:.4f}")
print(f"Test  RMSE : {test_rmse:.4f}")
print(f"Train R²   : {train_r2:.4f}")
print(f"Test  R²   : {test_r2:.4f}")
print("=============================================\n")

# ================================
# 8. Log Everything to MLflow
# ================================
with mlflow.start_run(run_name="Final_Best_Model"):

    mlflow.sklearn.log_model(
        final_model,
        artifact_path="stacking_regressor_model"
    )

    mlflow.log_params(best_params)
    mlflow.log_metric("best_test_mae", study.best_value)

    # Train metrics
    mlflow.log_metric("train_mae", train_mae)
    mlflow.log_metric("train_rmse", train_rmse)
    mlflow.log_metric("train_r2", train_r2)

    # Test metrics
    mlflow.log_metric("test_mae", test_mae)
    mlflow.log_metric("test_rmse", test_rmse)
    mlflow.log_metric("test_r2", test_r2)

print("Final model + metrics logged to MLflow successfully.")

Final model trained successfully.

========== FINAL MODEL PERFORMANCE ==========
Train MAE  : 2.8830
Test  MAE  : 2.9840
Train RMSE : 3.5794
Test  RMSE : 3.6843
Train R²   : 0.8530
Test  R²   : 0.8434



2026/02/10 07:01:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning:

Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.



🏃 View run Final_Best_Model at: https://dagshub.com/Aryanupadhyay23/Zomato-Food-Delivery-Time-prediction.mlflow/#/experiments/9/runs/b94a8105bef84a4ea49022ac54ce0211
🧪 View experiment at: https://dagshub.com/Aryanupadhyay23/Zomato-Food-Delivery-Time-prediction.mlflow/#/experiments/9
Final model + metrics logged to MLflow successfully.


In [22]:
# best parameter value

best_params = study.best_params

best_params

{'model': 'DT',
 'max_depth_dt': 6,
 'min_samples_split_dt': 7,
 'min_samples_leaf_dt': 8}

In [23]:
# parameter value counts

study.trials_dataframe()["params_model"].value_counts()

params_model
DT     11
LR      6
KNN     3
Name: count, dtype: int64

In [24]:
# mean scores for each meta estimator type

study.trials_dataframe().groupby(by="params_model")['value'].mean().sort_values()

params_model
LR     2.992210
DT     3.181829
KNN    3.421424
Name: value, dtype: float64

In [25]:
# best score

study.best_value

2.976028401071301

In [28]:
# optimization history plot

optuna.visualization.plot_optimization_history(study)

In [29]:
# parallel coord plot

optuna.visualization.plot_parallel_coordinate(study,params=["model"])